In [13]:
%pip install --upgrade langchain openai -q
%pip install unstructured -q
%pip install unstructured[local-inference] -q
%pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2 -q
%apt-get install poppler-utils


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\AL chat\AppData\Local\Temp\pip-build-env-5cb79sif\overlay\Lib\site-pa

In [1]:

from tqdm import tqdm
import docx
import json
import os
import openai
import pinecone
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain


d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# Load config values
with open(r'../config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['EMBEDDINGS_MODEL']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']
 
PINECONE_API_KEY = '36dba5ad-f423-4f2f-99e9-a05763d4d7be'
PINECONE_API_ENV = 'gcp-starter'

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Load your data

In [4]:
directory = './data/'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
len(documents)


1

In [5]:
def split_docs(documents, chunk_size=3000, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

15


In [6]:
embeddings = OpenAIEmbeddings(engine=deployment_name,model_name ="Text-embedding-ada-002")

query_result = embeddings.embed_query("Hello world")
len(query_result)


d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(
d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! model_name is not default parameter.
                    model_name was transferred to model_kwargs.
                    Please confirm that model_name is what you intended.
  warnings.warn(


1536

In [7]:
%pip install pinecone-client -q


Note: you may need to restart the kernel to use updated packages.


In [8]:
pinecone.init(
    api_key="36dba5ad-f423-4f2f-99e9-a05763d4d7be",
    environment="gcp-starter"
)

index_name = "demoineegpt"

index = Pinecone.from_documents(docs, embeddings, index_name=index_name)


In [9]:
def get_similiar_docs(query, k=2, score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query, k=k)
  else:
    similar_docs = index.similarity_search(query, k=k)
  return similar_docs


In [22]:
# model_name = "text-davinci-003"
model_name = "gpt-3.5-turbo"
#model_name = "gpt-4"
llm = OpenAI(engine = "demolabsmsneards")

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
  similar_docs = get_similiar_docs(query)

  #answer = chain.run(input_documents=similar_docs, question=query)

  print(similar_docs)
  answer = chain.run(input_documents=similar_docs, question=query)
  return answer


In [23]:
query = "what was the JPMorgan Chase Revenue in 2012?"
answer = get_answer(query)
print(answer)

# query = "How have relations between India and the US improved?"
# answer = get_answer(query)
# print(answer)


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2022-12-01 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


[Document(page_content='After the tumultuous changes in the investment banking business follow- ing the financial crisis of 2007 through 2009, the entire landscape changed. Following the banking crisis, investment banks needed capital. Some of the most storied investment banks, unable to raise money, merged with other banks or became commercial banks themselves. Suddenly, the financial system was comprised of behemoth banks that have the deposit-taking abili- ties of banks but also engage in investment banking. The result is the forma- tion of several mega-institutions that many people fear are “too big to fail,” including the ones shown in Table 1-1 .\n\nTable 1-1 Firm\n\nAmong the Last Banks Standing\n\n2012 Revenue ($ billions)\n\nJPMorgan Chase\n\n91.7\n\nWells Fargo\n\n79.5\n\nBank of America Merrill Lynch\n\n75.2\n\nCitigroup\n\n59.3\n\nGoldman Sachs\n\n34.2\n\nMorgan Stanley Source: S&P Capital IQ ( www.capitaliq.com )\n\n26.1\n\n05_9781118615775-ch01.indd 14\n\n05_9781118615775